# Preview Analysis data

This script surfaces analysis data to ascertain its value

In [5]:
import bote 
import json
import pandas as pd

conn, cursor = bote.db_connect()

cursor.execute("SELECT * FROM ask where analysis is not null")
data = cursor.fetchall()

#print(data)

df = pd.DataFrame(data)
df.columns = [desc[0] for desc in cursor.description]

df.head()


invalid_rows = []
for i, val in df['analysis'].items():
    try:
        json.loads(val)
    except (json.JSONDecodeError, TypeError):
        invalid_rows.append(i)

df = df.drop(invalid_rows)

df['analysis'] = df['analysis'].apply(json.loads)
df_analysis = pd.json_normalize(df['analysis'])
df = df.drop("embedding", axis=1)

def print_value_counts():
    for column in df_analysis.columns:
        print(f"\nColumn: {column}")
        print(df_analysis[column].value_counts())

def print_df():
    pd.set_option('display.max_colwidth', 1000)
    pd.set_option('display.max_rows', None)
    print(df)
    
#html_output = df.to_html()
#output_file_path = "output.html"
#with open(output_file_path, 'w') as f:
#    f.write(html_output)

df.head()
    
#df_analysis['advice_type'].unique()
#df_analysis['advice_type'].value_counts()

SyntaxError: 'return' outside function (99695582.py, line 16)

# 

In [41]:
import bote 
import json
import pandas as pd

conn, cursor = bote.db_connect()

cursor.execute("SELECT * FROM ask where analysis is not null order by RANDOM() limit 1")
data = cursor.fetchone()

analysis = data['analysis']
choices = analysis['choices']
if choices[0]['finish_reason'] == 'function_call':
    print(choices[0]['message']['function_call']['arguments'])


#analysis = json.loads(data['analysis'])
#print(json.dumps(analysis, indent=5))

{
  "advice_type": "OTHER",
  "subject": ["relationships", "sexuality and sex"]
}


In [54]:
import bote 
import json
import pandas as pd

conn, cursor = bote.db_connect()

cursor.execute("SELECT prompt, analysis FROM ask where analysis is not null order by RANDOM() limit 1")
data = cursor.fetchone()
print(data)

['I’m a freshman in college about three hours from my parents. I have 10 younger siblings—three biological and seven adopted from the foster care system. My parents are doing a wonderful service to these children in providing them with a warm, loving home. However, as the oldest child, a lot of responsibility was put on me from ages 12 to 18 to babysit, watch, and care for my younger siblings. I never felt like I had much of an opportunity to be myself or build my own interests because the vast majority of my time was occupied caring for children. I am attending college on an academic scholarship and I am enjoying my studies and learning to grow on my own. I recently got a summer job near my university and there is an affordable apartment I can rent for the summer while I am working. When I told my parents, they got very upset, claiming that I was “abandoning” them. They said that they needed me home to drive children to appointments and provide summer care. I feel like I gave my teena

In [1]:
import bote 
import json
import pandas as pd

conn, cursor = bote.db_connect()

cursor.execute("SELECT prompt, analysis FROM ask where analysis is not null")
data = cursor.fetchall()

# Convert the list of lists to a DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# Convert the DataFrame to an HTML table with Bootstrap styling
html_table = df.to_html(classes='table table-striped table-bordered')

# Create a complete HTML page
html_page = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Table from List of Lists</title>
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
</head>
<body>
    <div class="container mt-5">
        {html_table}
    </div>
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
</body>
</html>
"""

# Save the HTML page to a file
with open('output.html', 'w') as file:
    file.write(html_page)

print("HTML table saved to 'table.html'")


HTML table saved to 'table.html'
